##### **Setup Env & Lib imports**

In [101]:
# Pandas To Read File and get them in dataframe
import pandas as pd

# Numpy Libraries to work and get statistics of numeric values
import numpy as np

# Library to plot data and visualize our work
import matplotlib.pyplot as plt

# Library Seaborn to plot and visualize
import seaborn as sns
sns.set_style('whitegrid')

# Dash to build our server and get dashboard
from dash import Dash, html, dcc

# Dynamic Input and Output for Our components
from dash.dependencies import Input, Output, State

# Bootstrap Component to make responsive dashboard
import dash_bootstrap_components as dbc

# Dash Exceptions
from dash.exceptions import PreventUpdate

# To make figures and get graphs
import plotly.express as px
import plotly.graph_objs as go

# Data and Time Properties 
from datetime import date
from datetime import datetime

##### **Read Data and Preprocess it**

In [102]:
df = pd.read_csv("dataset/video_games_sales.csv")
df = df.iloc[:, 1:]

drop_row_index = df[df['Year'] > 2015].index
df = df.drop(drop_row_index)

df.shape

(16250, 10)

##### **Data Description**
>  1.  Rank: is just index for data (Dropped)
>  2.  Name: is the name of the game
>  3.  Platform: is the console game played on
>  4.  Year: the year of publishing ( > 2015 -> Dropped)
>  5.  Genre: is the type of game
>  6.  Publisher: Game publisher
>  7.  NA_Sales: Video game sales in North America (in millions)
>  8.  EU_Sales: Video game sales in Europe (in millions)
>  9.  JP_Sales: Video game sales in Japan (in millions)
>  10. Other_Sales: Video game sales in other countries (in millions)	
>  11. Global_Series: Total of worldwide sales

In [103]:
print(list(df.columns))
print(df.count())
print("Number Of Records: ", len(df), "record")

['Name', 'Platform', 'Year', 'Genre', 'Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
Name            16250
Platform        16250
Year            15979
Genre           16250
Publisher       16194
NA_Sales        16250
EU_Sales        16250
JP_Sales        16250
Other_Sales     16250
Global_Sales    16250
dtype: int64
Number Of Records:  16250 record


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16250 entries, 0 to 16597
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16250 non-null  object 
 1   Platform      16250 non-null  object 
 2   Year          15979 non-null  float64
 3   Genre         16250 non-null  object 
 4   Publisher     16194 non-null  object 
 5   NA_Sales      16250 non-null  float64
 6   EU_Sales      16250 non-null  float64
 7   JP_Sales      16250 non-null  float64
 8   Other_Sales   16250 non-null  float64
 9   Global_Sales  16250 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1.4+ MB


In [105]:
df.isna().sum().sort_values(ascending=False)

Year            271
Publisher        56
Name              0
Platform          0
Genre             0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [106]:
numeric_cols = list(df.select_dtypes(exclude='object').columns)
print(df[numeric_cols].isna().sum())

Year            271
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64


##### **Figures**

> ###### **Figure 1**

In [107]:
x_min = min(list(df.groupby(['Year']).sum().index))
x_max = max(list(df.groupby(['Year']).sum().index))
fig1 = px.line(df, x=np.array(np.arange(x_min, x_max+1)), 
            y=df.groupby(['Year'])['Global_Sales'].sum(),
            range_x=[1980, 2015], labels={
                "x" : "Year",
                "y" : "Sales (in millions)"
            })
fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
            y=df.groupby(['Year'])['NA_Sales'].sum(), name="North America")
fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
            y=df.groupby(['Year'])['JP_Sales'].sum(), name="Japan sales")
fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
            y=df.groupby(['Year'])['Other_Sales'].sum(), name="Other sales")
fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
            y=df.groupby(['Year'])['EU_Sales'].sum(), name="Europe sales")
fig1 = fig1.update_traces(line_width=2)
fig1 = fig1.update_layout(
    title={
        'text': "Video Games Sales across Years in Different Countries",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig1 = fig1.update_traces(mode="markers+lines", hovertemplate=None)
fig1 = fig1.update_layout(hovermode="x unified")
fig1 = fig1.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across")
fig1 = fig1.update_yaxes(showspikes=True, spikecolor="orange", spikethickness=2)
fig1 = fig1.update_layout(spikedistance=1000, hoverdistance=100)
fig1 = fig1.update_xaxes(tickangle=45)
fig1 = fig1.update_xaxes(tick0=1980, dtick=1)
fig1 = fig1.update_yaxes(tick0=0, dtick=50)
fig1 = fig1.update_layout({
'plot_bgcolor': 'white',
'paper_bgcolor': 'white',
})
fig1 = fig1.update_layout(title_font_color="#444", font_color="#444")

fig1 = fig1.update_layout(font=dict(
        family= "Inter, sans-serif",
        size=18,
        color="#444"
))

fig1 = fig1.update_xaxes(gridcolor='#444')
fig1 = fig1.update_yaxes(gridcolor='#444')

> ###### **Figure 2**

In [108]:
data_sales = df.groupby(['Platform'])[['Global_Sales', 'NA_Sales', 'JP_Sales', 'EU_Sales', 'Other_Sales']].sum().sort_values(by="Global_Sales", ascending=False).head(6)
pcharts = []
for row_ind in range(len(data_sales)):
        if data_sales.iloc[row_ind, 1] > 0.0 and data_sales.iloc[row_ind, 2] > 0.0 and data_sales.iloc[row_ind, 3] > 0.0 and data_sales.iloc[row_ind, 4] > 0.0:        
            fig = px.pie(data_sales, values=np.array([data_sales.iloc[row_ind, 1], data_sales.iloc[row_ind, 3], 
                                        data_sales.iloc[row_ind, 2], data_sales.iloc[row_ind, 4]]), names=['North America', 'Europe', 'Japan', 'Other'],hole=.4, width=500, height=400)
            colors = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
            fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                marker=dict(colors=colors, line=dict(color='#000000', width=2)))
            fig.update_layout(title_text="Descriptive Global Sales for "+ str(data_sales.index[row_ind]))
            fig = fig.update_layout({
            'plot_bgcolor': 'white',
            'paper_bgcolor': 'white',
            })
            fig = fig.update_layout(title_font_color="#444", font_color="#444")
            fig = fig.update_layout(font=dict(
                    family= "Inter, sans-serif",
                    size=18,
                    color="#444"))
            pcharts.append(fig)

> ###### **Figure 3**

In [109]:
dft = df.Platform.value_counts()
df_tmp = df[df.Platform.isin(dft[ dft > 500].index)]
figr = px.histogram(df_tmp, x="Genre", color="Platform", barmode="group", opacity=1, hover_data=df.columns)
figr = figr.update_xaxes(title_standoff = 25, showline=True, linewidth=2, linecolor='#666', mirror=True)
figr = figr.update_yaxes(title_text='Frequency', title_standoff = 25, showline=True,
                linewidth=2, linecolor='#666', mirror=True, gridcolor='LightGray')
figr = figr.update_layout(bargap=0.3, bargroupgap=0.2)
figr = figr.update_layout({'plot_bgcolor': 'white', 'paper_bgcolor': 'white',})
figr = figr.update_layout(title_font_color="white", font_color="white")
figr = figr.update_layout(font=dict(family= "Inter, sans-serif", size=18, color="white"))

> ###### **Figure 4** 

In [110]:
s='''
        lightskyblue, mediumorchid, maroon, lime,
        blue, black, red, olive, orange, silver, firebrick, magenta, ivory,
        cadetblue, yellow, violet, salmon, darkorange,
        coral, cornflowerblue,
        crimson, darkblue, peru, darkcyan, slategray,
        darkgoldenrod, royalblue, darkgray, darkgreen,
        saddlebrown, darkkhaki, darkmagenta, darkolivegreen,
        darkorchid, darkred, darkseagreen,
        darkslateblue, darkslategrey,
        darkturquoise, deepskyblue,
        dimgray, dodgerblue,
        forestgreen, fuchsia, gainsboro,
        gold, goldenrod, grey, green,
        greenyellow, honeydew, hotpink, indianred, indigo,
        khaki, lavenderblush, lawngreen,
        lemonchiffon, lightblue, lightcoral, lightcyan,
        lightgoldenrodyellow, lightgray, lightgrey,
        lightgreen, lightpink, lightsalmon, lightseagreen,
        lightslategray, lightslategrey,
        lightsteelblue, lightyellow, lime, limegreen,
        linen, mediumaquamarine,
        mediumblue, mediumpurple,
        mediumseagreen, mediumslateblue, mediumspringgreen,
        mediumturquoise, mediumvioletred, midnightblue,
        mintcream, mistyrose, moccasin, navajowhite, navy,
        oldlace, olivedrab, orangered,
        orchid, palegoldenrod, palegreen, paleturquoise,
        papayawhip, peachpuff, pink,
        plum, powderblue, rosybrown,
        sandybrown,
        seagreen, seashell, sienna, skyblue,
        slateblue, slategrey, snow, springgreen,
        steelblue, tan, teal, thistle, tomato, turquoise,
        wheat, white, whitesmoke,
        yellowgreen
        '''
names = s.replace('\n', '').replace(' ', '')
all_colors=names.split(',')

salesper = df.groupby(['Platform', 'Year'])['Global_Sales'].mean()
salesper = salesper.reset_index()

# set a unique color for each platform as a column in salesper dataframe
plat_colors = dict()

for colour, plat in zip(all_colors[:salesper.Platform.nunique()] ,salesper.Platform.unique()) :
        plat_colors[plat] = colour

salesper['plat_color'] = salesper.apply(lambda row : plat_colors[row['Platform']],axis = 1)

def racing_graph(df, col, x_lim = 4, y_lim = 5, y_siz = 14):
    years= sorted(list(df['Year'].unique()))

    n_frame= []

    for y in years:
        dataframe=df[(df['Year']==y)]
        dataframe=dataframe.nlargest(n=5,columns=['Global_Sales'])
        dataframe=dataframe.sort_values(by=['Year','Global_Sales'])

        n_frame.append(dataframe)
    

    fig = go.Figure(
        data=[
            go.Bar(
            x=n_frame[0]['Global_Sales'], y=n_frame[0][col],orientation='h',
            text=n_frame[0]['Global_Sales'], texttemplate='%{text:.3s}',
            textfont={'size':18}, textposition='inside', insidetextanchor='middle',
            width=0.9, marker = dict(color=n_frame[0]['plat_color']))

        ],
        layout=go.Layout(
            xaxis=dict(range=[0, x_lim], autorange=False, title=dict(text='Global sales (B)',font=dict(size=18))),
            yaxis=dict(range=[-1, y_lim], autorange=False, title=dict(text=col,font=dict(size=18)), tickfont=dict(size=y_siz)),
            title=dict(text='Global sales by {} starting 1980'.format(col),font=dict(size=28),x=0.5,xanchor='center'),
            updatemenus=[dict(
                type="buttons", direction= "left", pad = {"r": 5, "t": 5},
                x = 0,
                xanchor = "right",
                y = -0.1,
                yanchor = "top",
                buttons=[dict(label="Play", method="animate",

                            args=[None,
                            {"frame": {"duration": 2500, "redraw": True}, "transition": {"duration":250,
                            "easing": "linear"}}]
                ), {
                "args": [[None], {"frame": {"duration": 0, "redraw": False},
                                "mode": "immediate",
                                "transition": {"duration": 0}}],
                "label": "Stop",
                "method": "animate"
            }]
            )]
        ),
        frames=[
                go.Frame(
                    data=[
                            go.Bar(x=value['Global_Sales'], y=value[col],
                            orientation='h',text=value['Global_Sales'], marker = dict(
                        color=value['plat_color']))
                        ],
                    layout=go.Layout(
                            xaxis=dict(range=[0, x_lim], autorange=False),
                            yaxis=dict(range=[-1, y_lim], autorange=False, tickfont=dict(size=y_siz)),
                            title=dict(text='Global Sales by {}: {}'.format(col, str(value['Year'].values[0])),
                            font=dict(size=28))
                        )
                )
            for value in n_frame
        ]
    )
    fig = fig.update_layout({'plot_bgcolor': '#f0f0f0', 'paper_bgcolor': '#f0f0f0'})
    fig = fig.update_layout(title_font_color="black", font_color="black")
    fig = fig.update_xaxes(gridcolor='#444')
    fig = fig.update_yaxes(gridcolor='#444')
    
    return fig
    
genre_sales = df.groupby(['Genre', 'Year'])['Global_Sales'].mean()
genre_sales = genre_sales.reset_index()
gen_colors = dict()

for colour, plat in zip(all_colors[:genre_sales.Genre.nunique()] ,genre_sales.Genre.unique()) :
    gen_colors[plat] = colour
genre_sales['plat_color'] = genre_sales.apply(lambda row : gen_colors[row['Genre']],axis = 1)

##### **Layout**

In [111]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP,
                                "assets/dashboardV2.0.css",
                                "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.1.1/css/all.min.css"],
        external_scripts=["assets/jquery.min.js",
                    "assets/jquery.nicescroll.min.js",
                    "assets/dashboardV2.0.js",
                    "https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.1.1/js/all.min.js"])

app.layout = html.Div(
[
    html.Aside(
    [
        html.Div(
        [
            html.Img(className="rounded-pill img-fluid", width="65",
                    src="https://uniim1.shutterfly.com/ng/services/mediarender/THISLIFE/021036514417/media/23148907008/medium/1501685726/enhance",
                    alt=""),
            html.Div(
            [
                html.H5("Yasser Ashraf", className="fs-6 mb-0 text-decoration-none"),
                html.P("Lorem ipsum dolor sit amet consectetur.",className="mt-1 mb-0")
            ], className="ms-2")
        ], className="sidebar-header d-flex justify-content-center align-items-center px-3 py-4"),
        
        html.Div(
        [
            dbc.Input(type="text", className="form-control w-100 border-0 bg-transparent",
                    placeholder="Search here.")
        ], className="search position-relative text-center px-4 py-3 mt-2"),
    
        html.Ul(
        [
            html.Li(
            [
                html.A([
                    html.I(className="fa-solid fa-house-user"),
                    html.Label("Dashboard", className="p-2")
                ],href='.', className='a-list')
            ], className="list-li"),
            html.Li(
            [
                html.A([
                    html.I(className="fa-solid fa-table"),
                    html.Label("Data", className="p-2")
                ], href='#chart1', className='a-list')
            ], className="list-li"),
            html.Li(
            [
                html.A([
                    html.I(className="fa-solid fa-chart-simple"),
                    html.Label("Charts", className="p-2")
                ], href='#chart2', className='a-list')
            ], className="list-li")
        ], className="categories list-unstyled mt-2")
    ], className="sidebar position-fixed top-0 left-0 overflow-auto h-100 float-left", id="show-side-navigation1"),
    
    html.Section(
    [
        dbc.Navbar(
        [
            html.Div([
                dbc.Col([
                    html.H2("VGS", className="display-4 text-center navbar-brand p-3")
                ], width=2, className="menu-class"),
                
                dbc.Col([], width=6),
                dbc.Col([
                    html.Div([
                        html.A([
                            html.I(className="fas fa-user INav")
                        ], href="."),
                        html.A([
                            html.I(className="fas fa-cog fa-spin INav")
                        ], href="."),
                        html.A([
                            html.I(className="fa-solid fa-moon INav")
                        ], href="."),
                        html.A([
                            html.I(className="fa-solid fa-bars INav", id="menu-bars")
                        ])
                    ], className="icons"),
                ], width=4, className="right-col")
            ], className="row w-100 nav-row"),
        ], className="container-fluid bg-white border-navbar"),
        
        html.Div(
        [
            html.Div(
            [
                html.Div(
                [
                    html.H1("Welcome to Dashboard", className="fs-3"),
                    html.P("Hello Yasser Ashraf, welcome to your awesome dashboard!", className="mb-0")
                ], className="content rounded-3 p-3")
            ], className="welcome"),
            
            html.Section(
            [
                html.Div(
                        [
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(df['Global_Sales'].sum(), className="text-danger"),
                                            html.P("Total Sales", className="mb-0 text-danger")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-globe text-danger fa-3x"), className="align-self-center beans")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body beans-card")    
                            ], color="danger", outline=True, className="card-custom beans-card-custom"), width=3),
                            
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(df['NA_Sales'].sum(), className="text-success"),
                                            html.P("Total North America Sales", className="mb-0 text-success")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-earth-americas text-success fa-3x"), className="align-self-center beans")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")    
                            ], color="success", outline=True, className="card-custom beans-card-custom"), width=3),
                            
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(df['JP_Sales'].sum(), className="text-warning"),
                                            html.P("Total Japan Sales", className="mb-0 text-warning")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-earth-asia text-warning fa-3x"), className="align-self-center beans")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")
                            ], color="warning", outline=True, className="card-custom beans-card-custom"), width=3),
                            
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(df['EU_Sales'].sum(), className="text-info"),
                                            html.P("Total Europe Sales", className="mb-0 text-info")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-earth-europe text-info fa-3x"), className="align-self-center beans")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")    
                            ], color="info", outline=True, className="card-custom beans-card-custom"), width=3)
                        ], className="row w-100 container-fluid bins-container"),
            ], className="statistics mt-4"),
            
            html.Section(
            [
                html.Div([
                    dbc.Col(dcc.Graph(figure=fig1, id="graph1"), width=12) ], className="row graph-section1 container-fluid p-2"),
                    
                    html.Div([
                        dbc.Col([
                            html.Div([
                                dbc.Col([
                                    html.P("Year",
                                        className="year-label"),
                                ], width= 2),
                                dbc.Col([
                                    dcc.DatePickerRange(
                                        id='my-date-picker-range',
                                        min_date_allowed=date(int(min(df['Year'])), 1, 1),
                                        max_date_allowed=date(int(max(df['Year'])), 12, 31),
                                        display_format='YYYY',
                                        start_date_placeholder_text='YYYY',
                                        day_size=32,
                                        start_date=date(int(min(df['Year'])), 1, 1),
                                        end_date=date(int(max(df['Year'])), 12, 31),
                                        clearable=True,
                                        style={'zIndex': 10},
                                        show_outside_days=False,
                                    )
                                ], width=10)
                            ], className="row graph-config mt-2"),
                        ], width=5),
                        
                        dbc.Col([
                            html.Div([
                                dbc.Col([
                                    html.P("Graphs", className="year-label")
                                ], width=2),
                                
                                dbc.Col([
                                    dcc.Checklist(
                                        options=[
                                            {'label' : 'Europe','value' : 'EU_Sales'},
                                            {'label' : 'North America','value' : 'NA_Sales'},
                                            {'label' : 'Japan and Asia','value' : 'JP_Sales'},
                                            {'label' : 'Other Countries','value' : 'Other_Sales'}
                                        ],
                                        value=['EU_Sales', 'NA_Sales', 'JP_Sales', 'Other_Sales'],
                                        inline=True,
                                        labelStyle={'font-size': '20px', 'padding-left': '15px', 'display': 'block'},
                                        id='sales-checklist'
                                    )
                                ], width=10)
                            ],className="row w-100 mt-2")
                        ], width=7),
                    ], className="row w-100 container-fluid main-control")
            ], className="charts mt-4", id='chart1'),
            
            html.Section(
            [
                html.Div(
                        [
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(len(df), className="text-danger"),
                                            html.P("Games", className="mb-0 text-danger")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-gamepad text-danger fa-3x"), className="align-self-center")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")    
                            ], color="danger", outline=True, className="card-custom"), width=3),
                            
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(len(df['Genre'].unique()), className="text-success"),
                                            html.P("Game Categories", className="mb-0 text-success")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-trophy text-success fa-3x"), className="align-self-center")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")    
                            ], color="success", outline=True, className="card-custom"), width=3),
                            
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(len(df['Platform'].unique()), className="text-warning"),
                                            html.P("Console", className="mb-0 text-warning")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-terminal text-warning fa-3x"), className="align-self-center")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")
                            ], color="warning", outline=True, className="card-custom"), width=3),
                            
                            dbc.Col(dbc.Card([
                                html.Div([
                                    html.Div([
                                        html.Div([
                                            html.H3(len(df['Publisher'].unique()), className="text-info"),
                                            html.P("Different Companines", className="mb-0 text-info")
                                        ]),
                                        html.Div(html.I(className="fa-solid fa-building text-info fa-3x"), className="align-self-center")
                                    ], className="d-flex justify-content-between px-md-1")
                                ], className="card-body")    
                            ], color="info", outline=True, className="card-custom"), width=3)
                        ], className="row w-100 container-fluid bins-container"),
            ], className="statistics mt-4"),
            
            html.Section(
            [
                html.Div([
                    dbc.Col([dcc.Graph(id="graph12", figure=pcharts[0])], width=4),
                    dbc.Col([dcc.Graph(id="graph22", figure=pcharts[1])], width=4),
                    dbc.Col([dcc.Graph(id="graph32", figure=pcharts[2])], width=4),
                    dbc.Col([dcc.Graph(id="graph42", figure=pcharts[3])], width=4),
                    dbc.Col([dcc.Graph(id="graph52", figure=pcharts[4])], width=4),
                    dbc.Col([dcc.Graph(id="graph62", figure=pcharts[5])], width=4)
                ], className="row container-fluid w-100"),
            ], className="charts mt-4", id="pieCharts"),
            
            html.Section(
            [
                html.Div([
                    dbc.Col([
                        html.Div(dbc.Button("next", className="btn btn-next", id="next-btn", n_clicks=0), className="w-100 temp")
                    ], width=12)
                ], className="row container w-100")
            ], className="charts mt-4", id='chart2')
            
        ], className="p-4"),
        
        html.Section(
        [
            html.Div(
            [
                dbc.Col([
                    html.H1('Animated Genre and Platform global sales over years', style = {"color":"#444"}, className='text-center'),
                    html.Br(),
                    html.H4("Select an animation:", style = {"color":"#444"}),
                    dcc.RadioItems(
                        id='selection',
                        options=["Platform", "Genre"],
                        value='Platform',
                        style={"color": "#555"}
                    ),
                    html.Br(),
                    dcc.Loading(dcc.Graph(id="graph"), type="cube")    
                ], width=12)
            ],className="row w-100 container-fluid")
        ], className="charts mt-4 mb-2"),
        
        html.Footer(
        [
            html.Div(
            [
                dbc.Col([html.Label("vgs.analysis@hotmail.com", style={"color": "#666"})], width=3),
                dbc.Col([html.Label("Created by Yasser Ashraf and Amr Samaha", style={"color": "#666"})], width=6),
                dbc.Col(
                [
                    html.I(className="fa-brands fa-facebook p-1", style={"color": "#4267B2"}),
                    html.I(className="fa-brands fa-instagram p-1", style={"color": "#405DE6"}),
                    html.I(className="fa-brands fa-twitter p-1", style={"color": "#1DA1F2"}),
                ], width=3)
            ], className="row conrainer-fluid w-100 footer p-3 ml-2")
        ]
        )
        
    ], id="wrapper"),
    
],className="container-fluid p-0 m-0 w-100")

@app.callback(
    Output("graph1", "figure"),
    Input("my-date-picker-range", "start_date"),
    Input("my-date-picker-range", "end_date"),
    Input("sales-checklist", "value")
)
def year_change(start, end, values):
    if start is None:
        start = date(1980, 1, 1)
    elif end is None:
        end = date(2015, 12, 31)
    elif start is None and end is None:
        start = date(1980, 1, 1)
        end = date(2015, 12, 31)
    
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    start_d = start.year
    end_d = end.year
    
    if end_d <= start_d:
        end_d = 2015
    
    df_temp = df.query('Year <=' +  str(end_d) + 'and Year >=' + str(start_d))
    
    x_min = min(list(df_temp.groupby(['Year']).sum().index))
    x_max = max(list(df_temp.groupby(['Year']).sum().index))
    
    fig1 = px.line(df_temp, x=np.array(np.arange(x_min, x_max+1)), 
                y=df_temp.groupby(['Year'])['Global_Sales'].sum(),
                range_x=[start_d, end_d], labels={
                    "x" : "Year",
                    "y" : "Sales (in millions)"
                })
    if "NA_Sales" in values:
        fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
                    y=df_temp.groupby(['Year'])['NA_Sales'].sum(), name="North America")
    if "JP_Sales" in values:
        fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
                    y=df_temp.groupby(['Year'])['JP_Sales'].sum(), name="Japan sales")
    if "Other_Sales" in values:
        fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
                    y=df_temp.groupby(['Year'])['Other_Sales'].sum(), name="Other sales")
    if "EU_Sales" in values:
        fig1.add_scatter(x=np.array(np.arange(x_min, x_max+1)), 
                    y=df_temp.groupby(['Year'])['EU_Sales'].sum(), name="Europe sales")
        
    fig1 = fig1.update_traces(line_width=2)
    fig1 = fig1.update_layout(
        title={
            'text': "Video Games Sales across Years in Different Countries",
            'y':0.96,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    fig1 = fig1.update_traces(mode="markers+lines", hovertemplate=None)
    fig1 = fig1.update_layout(hovermode="x unified")
    fig1 = fig1.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across")
    fig1 = fig1.update_yaxes(showspikes=True, spikecolor="orange", spikethickness=2)
    fig1 = fig1.update_layout(spikedistance=1000, hoverdistance=100)
    fig1 = fig1.update_xaxes(tickangle=45)
    fig1 = fig1.update_xaxes(tick0=1980, dtick=1)
    fig1 = fig1.update_yaxes(tick0=0, dtick=50)
    fig1 = fig1.update_layout({
    'plot_bgcolor': '#F3F4F6',
    'paper_bgcolor': '#F3F4F6',
    })
    fig1 = fig1.update_layout(title_font_color="#444", font_color="#444")

    fig1 = fig1.update_layout(font=dict(
            family= "Inter, sans-serif",
            size=18,
            color="#444"
    ))

    fig1 = fig1.update_xaxes(gridcolor='#888')
    fig1 = fig1.update_yaxes(gridcolor='#888')
    
    return fig1

@app.callback(
    Output('pieCharts', 'children'),
    Input("next-btn", "n_clicks"),
    prevent_initial_call=True
)
def getCharts(n):    
        data_sales = df.groupby(['Platform'])[['Global_Sales', 'NA_Sales', 'JP_Sales', 'EU_Sales', 'Other_Sales']].sum().sort_values(by="Global_Sales", ascending=False)
        if (6*(n+1)) > len(df):
            raise PreventUpdate
        else:
            data_sales = data_sales.iloc[(6*n):(6*(n+1)),:]
            pcharts = []
            for row_ind in range(len(data_sales)):
                    if data_sales.iloc[row_ind, 1] > 0.0 and data_sales.iloc[row_ind, 2] > 0.0 and data_sales.iloc[row_ind, 3] > 0.0 and data_sales.iloc[row_ind, 4] > 0.0:        
                        fig = px.pie(data_sales, values=np.array([data_sales.iloc[row_ind, 1], data_sales.iloc[row_ind, 3], 
                                                    data_sales.iloc[row_ind, 2], data_sales.iloc[row_ind, 4]]), names=['North America', 'Europe', 'Japan', 'Other'],hole=.4, width=500, height=400)
                        colors = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
                        fig.update_traces(hoverinfo='label+percent', textinfo='percent', textfont_size=20,
                            marker=dict(colors=colors, line=dict(color='#000000', width=2)))
                        fig.update_layout(title_text="Descriptive Global Sales for "+ str(data_sales.index[row_ind]))
                        fig = fig.update_layout({
                        'plot_bgcolor': 'white',
                        'paper_bgcolor': 'white',
                        })
                        fig = fig.update_layout(title_font_color="#444", font_color="#444")
                        fig = fig.update_layout(font=dict(
                                family= "Inter, sans-serif",
                                size=18,
                                color="#444"))
                        pcharts.append(fig)
        return html.Div([
                    dbc.Col([dcc.Graph(id="graph12", figure=pcharts[0])], width=4),
                    dbc.Col([dcc.Graph(id="graph22", figure=pcharts[1])], width=4),
                    dbc.Col([dcc.Graph(id="graph32", figure=pcharts[2])], width=4),
                    dbc.Col([dcc.Graph(id="graph42", figure=pcharts[3])], width=4),
                    dbc.Col([dcc.Graph(id="graph52", figure=pcharts[4])], width=4),
                    dbc.Col([dcc.Graph(id="graph62", figure=pcharts[5])], width=4)
                ], className="row container-fluid w-100")

@app.callback(
    Output("graph", "figure"), 
    Input("selection", "value"))
def display_animated_graph(selection):
    animations = {
        'Platform': racing_graph(salesper, 'Platform'),
        'Genre': racing_graph(genre_sales, 'Genre', 8, 5, 12),
    }
    return animations[selection]

app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [01/May/2022 09:20:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/dashboardV2.0.css HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/dashboardV2.0.css?m=1651389603.4444683 HTTP/1.1" 200 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/dashboard.css?m=1650418850.9181137 HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/jquery.min.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/jquery.nicescroll.min.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/dashboardV2.0.js HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/dashboardV2.0.js?m=1651214064.8284771 HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/dashboard.js?m=1650258197.6750927 HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20:34] "GET /assets/jquery.min.js?m=1589992002.973801 HTTP/1.1" 304 -
127.0.0.1 - - [01/May/2022 09:20